# Lesson 12: FRI Protocol (Query Phase)

In this final lesson, we'll examine the query phase of the FRI protocol, where the verifier checks proof correctness by requesting specific values and verifying their consistency across layers.

In [ ]:
import numpy as np
from dataclasses import dataclass
from typing import List, Tuple, Dict
import hashlib

@dataclass
class MerkleProof:
    """Merkle tree membership proof."""
    value: complex
    path: List[Tuple[bytes, bool]]  # (hash, is_right)

@dataclass
class QueryResponse:
    """Response to verifier's query."""
    index: int
    value: complex
    merkle_proof: MerkleProof
    related_values: Dict[int, complex]  # Related values from previous layer

class FRIVerifier:
    """Implements FRI protocol query phase."""
    
    def __init__(self, commitments: List['FRICommitment']):
        self.commitments = commitments
        self.domain_size = len(commitments[0].layer_values)
        
    def generate_query_indices(self, num_queries: int) -> List[int]:
        """Generates random indices for queries."""
        return np.random.choice(self.domain_size, num_queries, replace=False)
    
    def verify_merkle_proof(self, proof: MerkleProof, root: bytes) -> bool:
        """Verifies Merkle proof."""
        current_hash = hashlib.sha256(str(proof.value).encode()).digest()
        
        for sibling_hash, is_right in proof.path:
            if is_right:
                current_hash = hashlib.sha256(current_hash + sibling_hash).digest()
            else:
                current_hash = hashlib.sha256(sibling_hash + current_hash).digest()
                
        return current_hash == root
    
    def verify_consistency(self, response: QueryResponse, 
                          prev_layer_index: int, alpha: complex) -> bool:
        """Verifies consistency between layers."""
        # Get related values from previous layer
        even_value = response.related_values[prev_layer_index]
        odd_value = response.related_values[prev_layer_index + 1]
        
        # Calculate expected value
        x = np.exp(2j * np.pi * prev_layer_index / self.domain_size)
        expected = even_value + alpha * odd_value * x
        
        # Check equality with numerical precision
        return np.abs(expected - response.value) < 1e-10
    
    def verify_query(self, query_index: int, layer_index: int) -> bool:
        """Verifies one query."""
        commitment = self.commitments[layer_index]
        value = commitment.layer_values[query_index]
        
        # Create dummy proof for demonstration
        proof = MerkleProof(
            value=value,
            path=[(b'sibling', True)] * int(np.log2(self.domain_size))
        )
        
        # Verify Merkle proof
        if not self.verify_merkle_proof(proof, commitment.merkle_root):
            return False
        
        # Verify consistency between layers
        if layer_index > 0:
            prev_index = query_index * 2
            related_values = {
                prev_index: self.commitments[layer_index-1].layer_values[prev_index],
                prev_index+1: self.commitments[layer_index-1].layer_values[prev_index+1]
            }
            response = QueryResponse(query_index, value, proof, related_values)
            if not self.verify_consistency(response, prev_index, commitment.alpha):
                return False
                
        return True

## Demonstrating Query Phase

In [ ]:
# Import necessary components from previous lessons
from lesson_11_fri_commit import FRICommitter

# Create test data
domain_size = 16
x = np.linspace(0, 2*np.pi, domain_size)
initial_values = np.sin(x) + 1j * np.cos(x)

# Create commitments
committer = FRICommitter(initial_values, num_layers=3)
commitments = committer.generate_commitments()

# Create verifier
verifier = FRIVerifier(commitments)

# Perform several queries
num_queries = 4
query_indices = verifier.generate_query_indices(num_queries)

# Visualize verification process
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 10))

# Plot values with marked queries
for layer_idx, commitment in enumerate(commitments):
    plt.subplot(len(commitments), 1, layer_idx + 1)
    
    # Plot all values
    plt.plot(np.abs(commitment.layer_values), 'b-', label='Layer Values')
    
    # Mark queried points
    if layer_idx == 0:
        plt.plot(query_indices, 
                 np.abs(commitment.layer_values[query_indices]),
                 'ro', label='Queries')
    else:
        # For subsequent layers, queries at corresponding positions
        related_indices = query_indices // (2 ** layer_idx)
        plt.plot(related_indices,
                 np.abs(commitment.layer_values[related_indices]),
                 'ro', label='Related Points')
    
    plt.title(f'Layer {layer_idx}')
    plt.xlabel('Index')
    plt.ylabel('|Value|')
    plt.grid(True)
    plt.legend()

plt.tight_layout()
plt.show()

# Check queries
print("\nQuery Verification Results:")
for query_index in query_indices:
    print(f"\nQuery for index {query_index}:")
    for layer_idx in range(len(commitments)):
        layer_query_index = query_index // (2 ** layer_idx)
        is_valid = verifier.verify_query(layer_query_index, layer_idx)
        print(f"Layer {layer_idx}: {'✓' if is_valid else '✗'}")

## Analysis of Query Phase

1. **Query Structure**:
   - Random index selection
   - Related values between layers
   - Merkle proofs for each value

2. **Verifier Checks**:
   - Merkle proof verification
   - Layer consistency verification
   - Final polynomial check

3. **Security**:
   - Probabilistic nature of verification
   - Impossibility of forgery without detection
   - Error detection efficiency

## Conclusion

We have completed our study of the STARK protocol by examining all its key components:

1. **Execution Trace**:
   - Program state recording
   - Verification rules
   - Polynomial constraints

2. **Cryptographic Primitives**:
   - Merkle trees
   - Commitments
   - Membership proofs

3. **Optimizations**:
   - DEEP technique
   - FRI mixing
   - Efficient verification

This set of lessons provides practical understanding of how STARK works in RISC Zero, and how various components interact to create efficient and secure proofs of program execution.